In [1]:
# ===========================================
# KINDA EXPERIMENTAL - USED FOR EXPLORATION OF DATA - dont expect this to run as it is
# ===========================================

import pandas as pd
import numpy as np
import nltk

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from nltk.util import ngrams
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer, RegexpStemmer
from string import punctuation

from collections import Counter

In [2]:
porter = PorterStemmer()
wnl = WordNetLemmatizer() 
stop = stopwords.words('english')

stop.append("!")
stop.append(',')
stop.append('')
stop.append('=')
stop = list(stop)
re = RegexpStemmer('[0-9]+')

# Preprocess text
def preprocess_text(string):
    # Annoying things!
    string = string.replace("=", "")
    string = string.replace("-", "")
    string = string.replace("'", "")
    tokens = word_tokenize(str(string))
    # Punctuations
    tokens = [re.stem(token.lower()) for token in tokens if token not in punctuation and token not in stop]
    # Stopwords
    #tokens = [token for token in tokens if token not in stop]
    # Numbers
    #tokens = [re.stem(token) for token in tokens]
    # convert to lowercase
    #tokens = [token.lower() for token in tokens]
    return (tokens)

# Make unigrams and bygrams from a classified data 
def make_grams(classified_data):
    unigram_frequencies = Counter([])
    #bigram_frequencies = Counter([])
    
    sentances = classified_data["comment_text"]
    for sentance in sentances:
        unigrams = ngrams(sentance, 1)
        # bigrams = ngrams(sentance, 2)
        unigram_frequencies += Counter(unigrams)
        # bigram_frequencies += Counter(bigrams)
    return unigram_frequencies# , bigram_frequencies)

In [ ]:
#train = pd.read_csv('test.csv', index_col=0)
#train_labels = pd.read_csv('test_labels.csv', index_col=0)

# Process data!
#train["comment_text"] = train.comment_text.apply(preprocess_text)
#data = pd.concat([train, train_labels], axis=1)


data = pd.read_csv('train.csv', index_col=0)
data["comment_text"] = data.comment_text.apply(preprocess_text)

In [ ]:
# Method that gets documents that only belong to only one of the classes
def filter_comments(data, toxic = 0, severe_toxic = 0, obscene = 0, threat = 0, insult = 0, identity_hate = 0):
    condition = (data.toxic == toxic) & (data.severe_toxic == severe_toxic) & (data.obscene == obscene) & (data.threat == threat) & (data.insult == insult) & (data.identity_hate == identity_hate)
    return data[condition]
    
    
toxic_only = filter_comments(data, toxic = 1)
severe_toxic_only = filter_comments(data, severe_toxic = 1)
obscene_only = filter_comments(data, obscene = 1)
threat_only = filter_comments(data, threat = 1)
insult_only = filter_comments(data, insult = 1)
identity_hate_only = filter_comments(data, identity_hate = 1)
cleant_documents = filter_comments(data)

In [ ]:
# Method to print precenage of documents compared to larger subset
def partition(message, all_docs, subset_docs):
    return message + "\t" +  str(round(len(subset_docs) / len(all_docs), 6)) + "%\t ,count: " + str(len(subset_docs))


total_documents = len(data)
clean_document_count = len(cleant_documents)
toxic_document_count = len(toxic_only)
severe_toxic_document_count = len(severe_toxic_only)
obscene_document_count = len(obscene_only)
threat_document_count = len(threat_only)
insult_document_count = len(insult_only)
identity_hate_document_count = len(identity_hate_only)

print("Precentage of documents that belong to ONLY ONE class")
print("============")
print("Total documents: ", total_documents)
print(partition("Clean documents: \t", data, cleant_documents))
print("============")
print(partition("Only toxic documents: \t", data, toxic_only))
print(partition("Only severe toxic documents: ", data, severe_toxic_only))
print(partition("Only obscene documents: ", data, obscene_only))
print(partition("Only threath documents: ", data, threat_only))
print(partition("Only insult documents: \t", data, insult_only))
print(partition("Only identity documents: ", data, identity_hate_only))

In [ ]:
has_toxic = data[(data.toxic == 1)]
has_severe_toxic = data[(data.severe_toxic == 1)]
has_obscene = data[(data.obscene == 1)]
has_threat = data[(data.threat == 1)]
has_insult = data[(data.insult == 1)]
has_identity_hate = data[(data.identity_hate == 1)]


print("Precentage of documents that belong to a class")
print("============")
print("Total documents: ", total_documents)
print(partition("Clean documents: \t", data, cleant_documents))
print("============")
print(partition("Toxic documents: \t", data, has_toxic))
print(partition("Severe toxic documents: ", data, has_severe_toxic))
print(partition("Obscene documents \t: ", data, has_obscene))
print(partition("Threat documents: \t", data, has_threat))
print(partition("Insult documents: \t", data, has_insult))
print(partition("Identity hate documents: ", data, has_identity_hate))

( total_documents - clean_document_count ) / total_documents

In [ ]:
def filter_comments(data, condition):
    return data[condition]

classes = {}
classes["Toxic"] = (data.toxic == 1)
classes["SevereToxic"] = (data.severe_toxic == 1)
classes["Obscene"] = (data.obscene == 1)
classes["Threat"] = (data.threat == 1)
classes["Insult"] = (data.insult == 1)
classes["IdentityHate"] = (data.identity_hate == 1)

class_library = ["Toxic", "SevereToxic", "Obscene", "Threat", "Insult", "IdentityHate"]
freq_matrix = np.zeros((len(classes), len(classes)))
conditional_appearence = np.zeros((len(classes), len(classes)))
co_occurance = np.zeros((len(classes), len(classes)))

# How frequent classes are together?
counter1 = 0
counter2 = 0
for class1 in class_library:
    counter2 = 0
    for class2 in class_library:
        condition = classes[class1] & classes[class2]
        common_elements = filter_comments(data, condition)
        freq_matrix[counter1,counter2] =  round(len(common_elements)  / (len(data) - len(cleant_documents)), 3)
        counter2 += 1
    counter1+= 1
        
# How many of the classes co-occur if one occurs?
counter1 = 0
counter2 = 0
for class1 in class_library:
    counter2 = 0
    class_elements = filter_comments(data, classes[class1])
    for class2 in class_library:
        condition2 = classes[class1] & classes[class2]
        common_elements = filter_comments(data, condition2)
        conditional_appearence[counter1,counter2] =  round(len(common_elements)  / (len(class_elements)), 3)
        counter2 += 1
    counter1+= 1

counter1 = 0
counter2 = 0
for class1 in class_library:
    counter2 = 0
    class1_elements = filter_comments(data, classes[class1])
    for class2 in class_library:
        class2_elements = filter_comments(data, classes[class2])
        condition2 = classes[class1] & classes[class2]
        common_elements = filter_comments(data, condition2)
        co_occurance[counter1,counter2] =  round(len(common_elements)  / (len(class1_elements) + len(class2_elements)), 3)
        counter2 += 1
    counter1+= 1

print("\n")
    
print(freq_matrix)

print("\n")

print(conditional_appearence)

print("\n")

print(co_occurance)

In [ ]:
# Calculate how many words comments in certain class have
def calculate_avarage_words(dataframe):
    words = 0
    for comment in dataframe["comment_text"]:
        words += len(comment)
    
    return round(words / len(dataframe))


print("Avarage ammount words per comment")
print("=======")
print("Clean docs: \t", calculate_avarage_words(cleant_documents))
print("=======")
print("Toxic docs: \t", calculate_avarage_words(has_toxic))
print("Severe Toxic: \t", calculate_avarage_words(has_severe_toxic))
print("Obscene docs: \t", calculate_avarage_words(has_obscene))
print("Threat docs: \t", calculate_avarage_words(has_threat))
print("Insult docs: \t", calculate_avarage_words(has_insult))
print("Hate docs: \t", calculate_avarage_words(has_identity_hate))

In [ ]:
cleant_documents

In [ ]:
insult_unigram = make_grams(cleant_documents)

insult_unigram.most_common(10)

# Testing NN with BOW or something!

In [3]:
import numpy as np
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd
from torch.autograd import Variable
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

torch.manual_seed(1)

print('number of CUDA devices: ',torch.cuda.device_count())
print(torch.cuda.get_device_name(0))
device = torch.cuda.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

number of CUDA devices:  1
Quadro P5000


In [4]:
train = pd.read_csv('train.csv', index_col=0)

X = train.comment_text.apply(preprocess_text)
Y = train.iloc[:, 1:]

In [5]:
# Make naive BOW-presentation
word_to_ix = {}
for sent in X:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)


In [37]:
def make_bow_vector(sentence, word_to_ix):
    vec = np.zeros(len(word_to_ix))
    for word in sentence:
        vec[word_to_ix[word]] += 1
    return vec

def make_target(label):
    tmp = []
    for lab in label:
        tmp.append(lab)
    return tuple( np.array(tmp) )

class BoWClassifier(nn.Module):  # inheriting from nn.Module!

    def __init__(self, num_labels, vocab_size, n_hidden1, n_hidden2):
        super(BoWClassifier, self).__init__()
        self.hidden1 = nn.Linear(vocab_size, n_hidden1)
        self.hidden2 = nn.Linear(n_hidden1, n_hidden2)
        self.out = nn.Linear(n_hidden2, num_labels)

    def forward(self, x_val):
        x = Variable(x_val, requires_grad=False)
        x = F.relu(self.hidden1(x))
        x = F.relu(self.hidden2(x))
        x = F.sigmoid(self.out(x))
        return x
    
class LSTMClassifier(nn.Module):
    
    def __init__(self, embedding_dim, num_labels, vocab_size, hidden_size):
        super(LSTMClassifier, self).__init__()
        self.hidden_dim = hidden_size
        self.lstm = nn.LSTM(vocab_size, hidden_size)
        self.hidden2label = nn.Linear(hidden_size, num_labels)
        self.hidden = (Variable(torch.zeros(1, 1, hidden_size)),
                        Variable(torch.zeros(1, 1, hidden_size)))
        
    def forward(self, x):
        lstm_out, self.hidden = self.lstm(x.view(len(x),1, -1), self.hidden)
        y  = self.hidden2label(lstm_out[-1])
        probs = F.sigmoid(y)
        return probs
    

In [47]:
VOCAB_SIZE = 100 # len(word_to_ix) # 100
HIDDEN1 = 8
NUM_LABELS = 6

model = LSTMClassifier(1, NUM_LABELS, VOCAB_SIZE, HIDDEN1)
optimizer = optim.Adam(model.parameters())

epochs = 5
batch_size = 1
num_batches = int(len(X) * 0.9) // batch_size
critize = nn.BCELoss()

X_train = X[:int(len(X) * 0.95)]
y_train = Y[:int(len(X) * 0.95)]

X_test = X[int(len(X) * 0.95) + 1 :]
y_test = Y[int(len(X) * 0.95) + 1 :]
for epoch in range(epochs):
    losses = []
    for batch in range(500):
        start, end = batch * batch_size, (batch + 1) * batch_size
        X_batch = X_train[start:end]
        y_batch = y_train[start:end]
        
        model.zero_grad()

        test_x = np.array([d2v_model.infer_vector(comment) for comment in X_batch ])
        test_y = np.array([make_target(Y.iloc[i]) for i in range(len(y_batch)) ])

        #test_x = np.array([make_bow_vector(X[0], word_to_ix)])
        #test_y = np.array([make_target(Y.iloc[0])])

        test_x = Variable(torch.from_numpy(test_x))
        test_y = Variable(torch.from_numpy(test_y).float())

        output = model.forward(test_x)
        loss = critize(output, test_y)
        loss.backward(retain_graph=True)
        optimizer.step()
        losses.append(loss.data.mean())

    print("AVG was %.3f" % np.mean(losses))

AVG was 0.268
AVG was 0.021
AVG was 0.009
AVG was 0.005
AVG was 0.003


In [ ]:
VOCAB_SIZE = len(word_to_ix)
HIDDEN1 = 32
HIDDEN2 = 16
NUM_LABELS = 6

model = BoWClassifier(NUM_LABELS, VOCAB_SIZE, HIDDEN1, HIDDEN2)
optimizer = optim.Adam(model.parameters())
# MultiLabelSoftMarginLoss - 
epochs = 5
batch_size = 1000
num_batches = int(len(X) * 0.9) // batch_size
# critize = nn.MultiLabelSoftMarginLoss()
critize = nn.BCELoss()


X_train = X[:int(len(X) * 0.95)]
y_train = Y[:int(len(X) * 0.95)]

X_test = X[int(len(X) * 0.95) + 1 :]
y_test = Y[int(len(X) * 0.95) + 1 :]
for epoch in range(epochs):
    losses = []
    for batch in range(num_batches):
        start, end = batch * batch_size, (batch + 1) * batch_size
        # Fetch the part of data neededVariable
        X_batch = X_train[start:end]
        y_batch = y_train[start:end]
        # Prepare data to BOW and make torch vectors
        X_batch = np.array([make_bow_vector(comment, word_to_ix) for comment in X_batch])
        y_batch = np.array([ make_target(y_batch.iloc[i]) for i in range(len(y_batch))])
        X_batch = torch.from_numpy(X_batch).float()
        y_batch = Variable(torch.from_numpy(y_batch).float(), requires_grad=False)
        # GD towards opt
        model.zero_grad()
        output_fw = model.forward(X_batch)

        loss = critize(output_fw, y_batch)
        loss.backward()
        optimizer.step()
        losses.append(loss.data.mean())

    X_batch = X_test
    y_batch = y_test
    X_batch = np.array([make_bow_vector(comment, word_to_ix) for comment in X_batch])
    y_batch = np.array([ make_target(y_batch.iloc[i]) for i in range(len(y_batch))])
    X_batch = torch.from_numpy(X_batch).float()
    # GD towards opt
    model.zero_grad()
    output_fw = model.forward(X_batch)
    score = roc_auc_score(y_batch, output_fw.data.numpy())

    print('[%d/%d] Training-error: %.3f' % (epoch+1, epochs, np.mean(losses)))
    print('[%d/%d] ROC-score:  %.3f' % (epoch+1, epochs, score))

In [ ]:
#!pip install --user gensim

In [7]:
import gensim

In [ ]:
def mean_of_comment_word2vec(comment, word_vectors):
    doc_vector = [np.sum(word_vectors[word]) for word in sentance if word in word_vectors.vocab]
    doc_vector = np.sum(doc_vector, axis=0)
    return doc_vector

# Word2Vec model
# Load google's w2v
word2vec = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True, limit=350000) 
word_vectors = word2vec.wv

sentance = X[0]
word_mean = mean_of_comment_word2vec(sentance, word_vectors)

In [ ]:
VOCAB_SIZE = 100
HIDDEN1 = 256
HIDDEN2 = 128
NUM_LABELS = 6

model = BoWClassifier(NUM_LABELS, VOCAB_SIZE, HIDDEN1, HIDDEN2)
optimizer = optim.Adam(model.parameters())

epochs = 10
batch_size = 1000
num_batches = int(len(X) * 0.8) // batch_size
critize = nn.MultiLabelSoftMarginLoss()
#critize = nn.BCELoss()


X_train = X[:int(len(X) * 0.8)]
y_train = Y[:int(len(X) * 0.8)]

X_test = X[int(len(X) * 0.8) + 1 :]
y_test = Y[int(len(X) * 0.8) + 1 :]

for epoch in range(epochs):
    losses = []
    for batch in range(num_batches):
        start, end = batch * batch_size, (batch + 1) * batch_size
        # Fetch the part of data needed
        X_batch = X_train[start:end]
        y_batch = y_train[start:end]
        # Prepare data to BOW and make torch vectors
        X_batch = np.array([d2v_model.infer_vector(comment) for comment in X_batch])
        y_batch = np.array([ make_target(y_batch.iloc[i]) for i in range(len(y_batch))])
        
        X_batch = torch.from_numpy(X_batch).float()
        y_batch = Variable(torch.from_numpy(y_batch).float(), requires_grad=False)
        # GD towards opt
        model.zero_grad()
        output_fw = model.forward(X_batch)

        loss = critize(output_fw, y_batch)
        loss.backward()
        optimizer.step()
        losses.append(loss.data.mean())
        
    X_batch = X_test
    y_batch = y_test
    X_batch = np.array([d2v_model.infer_vector(comment) for comment in X_batch])
    y_batch = np.array([ make_target(y_batch.iloc[i]) for i in range(len(y_batch))])
    X_batch = torch.from_numpy(X_batch).float()
    # GD towards opt
    model.zero_grad()
    output_fw = model.forward(X_batch)
    score = roc_auc_score(y_batch, output_fw.data.numpy())

    print('[%d/%d] Training-error: %.3f' % (epoch+1, epochs, np.mean(losses)))
    print('[%d/%d] ROC-score:  %.3f' % (epoch+1, epochs, score))

In [8]:
def read_corpus(X):
    for i, line in enumerate(X):
        line = " ".join(line)
        yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])

d2v_model = gensim.models.doc2vec.Doc2Vec(vector_size=100, min_count=2, epochs=55)

corpus = list(read_corpus(X))
d2v_model.build_vocab(corpus)
%time d2v_model.train(corpus, total_examples=d2v_model.corpus_count, epochs=d2v_model.epochs)

CPU times: user 17min 2s, sys: 59.9 s, total: 18min 2s
Wall time: 7min 54s


In [ ]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier

SVM_pl = Pipeline([
    #('vectorizer', CountVectorizer(binary=True)),
    ('clf', OneVsRestClassifier(LinearSVC()))
    ])

In [ ]:
kf = KFold(n_splits=5, shuffle=True)

scores_SVC = []

for train_idx, test_idx in kf.split(X):
    X_train, y_train = X[train_idx], Y.iloc[train_idx, :].values
    X_test, y_test = X[test_idx], Y.iloc[test_idx, :].values
    X_train = np.array([d2v_model.infer_vector(comment) for comment in X_train])
    X_test = np.array([d2v_model.infer_vector(comment) for comment in X_test])
    
    SVM_pl.fit(X_train, y_train)
    y_pred = SVM_pl.predict(X_test)
    score = roc_auc_score(y_test, y_pred)
    scores_SVC.append(score)
    print(score)
    
print('MEAN: {}'.format(np.mean(scores_SVC)) )


In [48]:
test_x = np.array([d2v_model.infer_vector(X[6])])
test_x = Variable(torch.from_numpy(test_x))

model.forward(test_x)

Variable containing:
1.00000e-03 *
  2.1355  2.2144  3.0743  2.2770  2.0038  2.8827
[torch.FloatTensor of size 1x6]